### Started July 2, 2018


This file starts by taking in the cached matlab file. This is bc there were issues with running the FFT using python. It was basically stalling out. 

This is basically a work around till I can find a better solution like loading matlab from python.

Later will write a shell script so that I can call the matlab file, then the python file. 

### To Do:

- Once we figure out how to do fft here in python, we should be able to work from here on.
- Rewrite everthing so the functions are very clear.
- Write up plots in python


In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import scipy.io as sio

In [2]:
data_subfolder = 'wigs_U000962'
data_file_name = 'GSM2971252_Escherichia_coli_WT_Rend_seq_5_exo_MOPS_comp_25s_frag_pooled_*_no_shadow.wig'
matlab_file_name = 'cache.mat'
genome_annotation_file_name = "U00096.2.faa"
genome_size = 4639675 # length of genome in base pairs

cache = 1



In [11]:
#Functions for data loading and parsing
def load_wig_data(subfolder, data_file_name, genome_len):
    """
    The purpose of this function is to go create a data file for all the 
    wig sequencing data. The data is is in the form of a text file (.wig) and contains position 
    (first column) and count (second column) data.
    
    This function will simply take in the data files, and put them into an array for use by downstream functions.
    
    Requires:

    -Subfolder where files are located.
    -Name of data file containing a wildcard in the place of the directional info for that sequencing run.
    
    Returns:
    
    - dense_data_array: a 3 dimensional array.
        First dimension: Based on seq file.
        Second dimension: counts per position
        All positions through the lenght of the genome are recorded.
    - file_paths: a list containing all of the file paths 
    - seq_directions: a list containing the sequencing direction of each file,
        in the order it was pulled to create all_data. The directions are either 3' or 
        5' in either forward or reverse direction.
    """
    #parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    data_directory = os.path.join(os.getcwd(), 'data', subfolder, data_file_name)
    file_paths = []
    dense_data_array = np.zeros((4, genome_len + 1, 1))
    count = 0
    for file_path in glob.glob(data_directory):
        file_paths.append(file_path)
        with open (file_path, 'rt') as f:
            for line in f:
                if line[0].isdigit():
                    x, y = line.split()
                    dense_data_array[count, int(x) - 1] = float(y)
        count += 1
    seq_directions = get_seq_direction(file_paths)
    return file_paths, dense_data_array, seq_directions


def get_seq_direction(file_names):
    """
    Purpose:
    - To extract from the file names the seq direction of the imported wig file. 
        This is to avoid having to hard-code the directionality in the order the files 
        are pulled, although the direction options do need to be hard-coded, and 
        need to be present in the file name.
    Requires:
    - List of all files that need to be evaluated (this list is output in the function
        load_wig_data)
    - List of all the possible directions to be evaluated
        This list of directions should be the same regardless of the 
        sample. In this function these directions are hard coded. See possible_directions.
    Returns:
    -Read direction as a list of strings based on the order the files were imported 
        and are consequently stored.
    - The positions are listed in the order that the files were pulled and extracted
    in the function load_wig_data(). So the positions in this list reflect the 
    order of that data.
    """
    possible_directions = ['5f', '5r', '3f', '3r']
    directions = []
    for name in file_names:
        for direction in possible_directions:
            if direction in name.lower():
                directions.append(direction)
    return directions

def simplify_genome_annotation(annotation_file_name):
    """
    The purpose of this function is to pull out genome annotation information from the 
    original .faa file.

    Requires:
        -.faa genome annotation file in /genome/ subdirectory.

    Returns:
        -List where for each gene the the locus tag, gene name, gene name,
            gene direction, start position, and end position are recorded
            For example:
            ['b0001', 'thrL', '+', 190.0, 255.0]

    Note:
        -The annotation file has to be annotated in a very specific format.
        For example:
        >[locus_tag=b0585] [gene=fes] [protein=enterobactin/ferric enterobactin esterase] [location=612038..613162]
        -If file is not in correct format, update code below, or restructure file.
    """
    #parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    file_path = os.path.join(os.getcwd(), 'genome', genome_annotation_file_name)
    line_marker = '>'
    annotation_list = []
    with open (file_path, 'rt') as in_file:  
        for line in in_file:
            if line_marker in line:
                annotation_list.append(parse_single_gene(line))
    return annotation_list

def parse_single_gene(single_line):
    """
    Single text line (corresponding to a single gene) originally extracted from gene 
    anotation file in the function simplify_genome_annotation(), and parses it to 
    give extract the anotation information.
    Requires:
        - Single gene text line extracted from the genome annotation file.
    
    Returns:
        -A list contining the locus tag, gene name, gene direction ('+' or '-')(as strings), 
            and gene start and stop for each gene.
    Note:
        - For get_gene_direction_info the data supplied are taken from
            gene_info_not_parsed[-1], since sometimes protein descriptions
            contain [] and will throw off indexing if choosing the third 
            position. But the location information is always stored last.
    """
    gene_info_not_parsed = splice_string_to_attributes(single_line)
    locus_tag = get_locus_tag(gene_info_not_parsed[0])
    gene_name = get_gene_name(gene_info_not_parsed[1])
    gene_direction, gene_start, gene_stop = get_gene_direction_info(gene_info_not_parsed[-1])
    return [locus_tag, gene_name, gene_direction, gene_start, gene_stop]  

def splice_string_to_attributes(single_line):
    """
    Requires:
        -A line of text from the genome annotation file, 
    Returns:
        - list containing the parts necessary for further down stream string
        splicing. 
        -At this point it simply separates information based on whether it is surrounded by square
        brackets.
    """
    split_string = re.split(r'[\[\]]', single_line)
    strip_list = [gene_data for gene_data in split_string if gene_data.strip()][1:]
    return strip_list

def get_locus_tag(locus_info):
    """
    Requires:
        -String in the form: locus_tag=b0586,
    Returns:
        -Locus tag (string)
    """
    locus = locus_info.split('=')[1]
    return locus

def get_gene_name(gene_info):
    """
    Requires:
        -String in the form: gene=ybdZ,
    Returns:
        -Gene name (string)
    Note:
    Requires that no more than a single gene name is assigned per gene. 
    """
    name = gene_info.split('=')[1]
    return name

def get_gene_direction_info(direction_info):
    """
    Requires:
        - Takes in a string in the form: location=417113..418408 or
        location=complement(414974..416176).
    Returns:
        -If is the complemnent returns the direction as '-', or 
        else returns '+'.
        -Returns the start and stop positions as floats.
    """
    start = [] 
    stop = []
    location_str = direction_info.split('=')[1]
    direction  = ['-' if '(' in location_str else '+'][0]
    
    if direction == "+":
        start, stop = location_str.split('..')
    else:
        indices = [location_str.find(i) for i in ['(', ')']]
        start, stop = location_str[indices[0]+1:indices[1]].split('..')
    return direction , float(start), float(stop)

def load_mat_data(file_name):
    
    cache_path = os.path.join(os.getcwd(), file_name)
    cached_data = sio.loadmat(cache_path)
    data = np.squeeze(cached_data['data']).T
    z_peak = np.squeeze(cached_data['z_peak']).T
    z_step = np.squeeze(cached_data['z_step']).T
    
    return data, z_peak, z_step

def get_read_direction(order):

    f_ind = []
    r_ind = []
    for count in range(0,len(order)):
        if 'f' in order[count]:
            f_ind.append(count)
        elif 'r' in order[count]:
            r_ind.append(count)
    
    return f_ind, r_ind

def group_data_by_direction(all_count_data, seq_dir):
    
    """
    The purpose of this function is to take in all the count data (counts per position on the genome)
    for each of the 4 seq directions (3r, 3f, 5f, 5r) along with a list (seq_dir) indicating the indices
    where the forward and reverse indices are found.
    
    The first element in seq_dir is a list containing the indices of the forward reads, the second
    element contians a list containing the indices of the reverse reads.
    
    The reads for each direction are then just added together so that that all the reads in
    a particular direction are combined. Forward and reverse reads are appened together in a list,
    with forward reads being first.
    """
    
    grouped_data = []    
    for direction in seq_dir:
        combined_data_per_direction = np.zeros((len(all_count_data[0])))
        for index in direction:
            combined_data_per_direction += all_count_data[index]
        grouped_data.append(combined_data_per_direction)

    return grouped_data

def split_gene_annotation_by_direction(gene_annotation_list):
    #Convert list to pandas array for easier working
    df = pd.DataFrame(gene_annotation_list, columns=['Locus', 'Symbol', 'Direction', 'Start', 'Stop'])
    #Pull out Forward and Reverse genes from the gene annotation.
    forward_genes = df.loc[df['Direction'] == '+'].values.tolist()
    reverse_genes = df.loc[df['Direction'] == '-'].values.tolist()

    return forward_genes, reverse_genes

def loc_no_reads(read_data, window, read_frac, genome_len):    
    # Read data is taken in first in forward, then in reverse...
    indices_no_reads = []
    full_bool_map = []
    for data in read_data:
        bool_map=[]
        for i in range(0, genome_len + 2 - window):
            #along the genome scan for regions with high read counts
            bool_map.append(len(np.where(data[i : i + window] > 0)[0]) > read_frac * window)
        bool_map = np.asarray(bool_map)
        #the boolean map records the indices where the reads are 0.
        indices_no_reads.append(np.where(bool_map == False)[0])
        full_bool_map.append(bool_map) 
    return indices_no_reads, full_bool_map

## Plot Functions:



In [4]:
if cache:
    all_data, z_peak, z_step = load_mat_data(matlab_file_name)
    data_order_mat = ['3f', '3r', '5f', '5r'] # This is the order of the data from matlab. 
else:
    #For this to work fully... would need to add all functions to get the fft values.
    all_file_names, all_data, seq_directions = load_wig_data(data_subfolder, data_file_name, genome_size)



genome_annotation = simplify_genome_annotation(genome_annotation_file_name)
forward_genes, reverse_genes = split_gene_annotation_by_direction(genome_annotation)

sequencing_direction_index = get_read_direction(data_order_mat)

In [5]:
forward_genes

[['b0001', 'thrL', '+', 190.0, 255.0],
 ['b0002', 'thrA', '+', 337.0, 2799.0],
 ['b0003', 'thrB', '+', 2801.0, 3733.0],
 ['b0004', 'thrC', '+', 3734.0, 5020.0],
 ['b0005', 'yaaX', '+', 5234.0, 5530.0],
 ['b0008', 'talB', '+', 8238.0, 9191.0],
 ['b0009', 'mog', '+', 9306.0, 9893.0],
 ['b0014', 'dnaK', '+', 12163.0, 14079.0],
 ['b0015', 'dnaJ', '+', 14168.0, 15298.0],
 ['b0016', 'insL1', '+', 15445.0, 16557.0],
 ['b0019', 'nhaA', '+', 17489.0, 18655.0],
 ['b0020', 'nhaR', '+', 18715.0, 19620.0],
 ['b0024', 'yaaY', '+', 21181.0, 21399.0],
 ['b0025', 'ribF', '+', 21407.0, 22348.0],
 ['b0026', 'ileS', '+', 22391.0, 25207.0],
 ['b0027', 'lspA', '+', 25207.0, 25701.0],
 ['b0028', 'fkpB', '+', 25826.0, 26275.0],
 ['b0029', 'ispH', '+', 26277.0, 27227.0],
 ['b0030', 'rihC', '+', 27293.0, 28207.0],
 ['b0031', 'dapB', '+', 28374.0, 29195.0],
 ['b0032', 'carA', '+', 29651.0, 30799.0],
 ['b0033', 'carB', '+', 30817.0, 34038.0],
 ['b0034', 'caiF', '+', 34300.0, 34695.0],
 ['b0041', 'fixA', '+', 4240

In [9]:
grouped_data = group_data_by_direction(all_data, sequencing_direction_index)

In [12]:
# Tunable Parameters for looking for TU blocks

window_size = 11
frac_reads = 0.5
gene_pad = 100;



In [13]:
#boolean mapped reads is shifts in the matlab code
indices_of_no_reads, bool_map = loc_no_reads(grouped_data, window_size, frac_reads, genome_size)



In [16]:
# convert boolean map to df to get the indices of starts and stops

bool_map[0]

array([False, False, False, ..., False, False, False], dtype=bool)

In [26]:
bool_df_fwd = pd.DataFrame(np.array(bool_map[0]), columns = ['Value'])
bool_df_fwd

,Value
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


In [47]:
start_stop_df = bool_df_fwd[bool_df_fwd['Value'].diff().fillna(True)]
start_stop_df

,Value
0,False
155,True
416,False
426,True
472,False
473,True
842,False
843,True
922,False
925,True


In [93]:
# Assumes there is no read at the first bp of the genome. Write a catch for the instance where 
# there is a count in the first position... 

start_ind = start_stop_df.index[start_stop_df['Value'] == True].tolist()
stop_ind = start_stop_df.index[start_stop_df['Value'] == False].tolist()
# this next part will throw an error if there is not a 0 in the index... need to check that there
# is one on the stop_list,then remove it.
stop_ind.remove(0) 

# start and stop indices should be the same length now. double check though before continuing.

In [97]:
stops_to_remove = []
starts_to_remove = []
for i in range(len(stop_ind) - 1):
    if stop_ind[i] + window_size >= start_ind[i+1]:
        stops_to_remove.append(stop_ind[i])
        starts_to_remove.append(start_ind[i+1])

In [105]:
condensed_starts = [x for x in start_ind if x not in starts_to_remove]
condensed_stops = [x for x in stop_ind if x not in stops_to_remove]

In [106]:
condensed_starts

[155,
 1686,
 1783,
 2862,
 2888,
 3558,
 3583,
 3971,
 5129,
 5348,
 5498,
 5671,
 8200,
 9284,
 9422,
 9601,
 9868,
 12057,
 16961,
 17020,
 17130,
 17697,
 17771,
 17988,
 18048,
 18105,
 18129,
 18158,
 18284,
 18343,
 18383,
 18495,
 18652,
 18764,
 18829,
 19009,
 19380,
 21392,
 21453,
 21495,
 21535,
 21570,
 21635,
 21703,
 21776,
 21825,
 22556,
 25790,
 26175,
 26275,
 26609,
 26946,
 27084,
 27190,
 27205,
 27222,
 27252,
 28246,
 28358,
 28514,
 28594,
 28616,
 28714,
 28769,
 28785,
 28818,
 28884,
 29042,
 29087,
 29160,
 29185,
 29216,
 29239,
 29264,
 29564,
 29757,
 29834,
 29924,
 29956,
 29980,
 30039,
 30122,
 30153,
 30243,
 30297,
 30333,
 30356,
 30428,
 30593,
 30625,
 30707,
 30859,
 31240,
 31355,
 31475,
 31876,
 32265,
 32523,
 33018,
 33196,
 33295,
 34075,
 49881,
 49906,
 49989,
 50207,
 50241,
 50345,
 50395,
 50429,
 50492,
 57641,
 57765,
 57800,
 58705,
 58929,
 58959,
 59037,
 59076,
 59108,
 59147,
 59181,
 59215,
 59229,
 59247,
 59277,
 70238,
 7

In [107]:
condensed_stops

[1673,
 1770,
 2850,
 2874,
 3543,
 3567,
 3958,
 5077,
 5134,
 5353,
 5502,
 5678,
 9249,
 9287,
 9425,
 9614,
 9871,
 15402,
 17002,
 17023,
 17134,
 17703,
 17781,
 17996,
 18051,
 18111,
 18130,
 18164,
 18285,
 18347,
 18439,
 18640,
 18692,
 18766,
 18831,
 19022,
 19382,
 21428,
 21476,
 21498,
 21554,
 21621,
 21684,
 21764,
 21802,
 22544,
 25777,
 26162,
 26260,
 26587,
 26919,
 27024,
 27175,
 27192,
 27208,
 27240,
 27279,
 28254,
 28368,
 28524,
 28604,
 28621,
 28757,
 28772,
 28791,
 28819,
 28890,
 29052,
 29095,
 29161,
 29204,
 29225,
 29249,
 29270,
 29697,
 29821,
 29910,
 29927,
 29964,
 29986,
 30043,
 30124,
 30172,
 30256,
 30301,
 30338,
 30371,
 30434,
 30604,
 30629,
 30712,
 30868,
 31250,
 31361,
 31485,
 31891,
 32274,
 32529,
 33019,
 33198,
 33301,
 34077,
 49888,
 49910,
 49995,
 50228,
 50277,
 50362,
 50400,
 50436,
 50503,
 57647,
 57775,
 57802,
 58711,
 58931,
 58988,
 59042,
 59081,
 59111,
 59151,
 59182,
 59217,
 59235,
 59258,
 59332,
 70257,
 

In [110]:
n_f_genes = len(forward_genes)
n_r_genes = len(reverse_genes)
f_read_start = []

In [ ]:
putative_tu_start = []
putative_tu_stop = []
count = 0
for f_gene in forward_genes:
    if condensed_starts[count] < f_gene[3]:
        putative_tu_start.append(condensed_starts[count])
    

In [114]:
condensed_starts[0]

155

In [113]:
condensed_stops[0]

1673

In [117]:
len(forward_genes)

2110

In [115]:
forward_genes[0][3]


190.0

In [116]:
forward_genes[0][4]

255.0

In [48]:
shifts = indices_of_no_reads[0]
n_genes = len(forward_genes) - 1
n_shifts = len(shifts)

In [49]:
# basically a direct port of the matlab code

# The values i am getting here are slightly different than in matlab.. but the general outline works for now... 
# worry about the differences more later.

real_starts = []
real_stops = []
gene = 0
pos = 0



for gene in range(n_genes + 1):
    
    #if the first gene 
    if gene == 0 and forward_genes[gene][3] < shifts[pos]:
        real_starts.append(0)
    else:
        real_starts.append(shifts[pos])
    
    while gene < n_genes and pos < n_shifts:
        if forward_genes[gene][4] > shifts[pos] + window_size:
            pos += 1
        elif shifts[pos] > forward_genes[gene + 1][3] - gene_pad:
            gene += 1
        else:
            temp_pos = pos
            next_start = forward_genes[gene + 1][3]
            while temp_pos < n_shifts and shifts[temp_pos + 1] < next_start:
                temp_pos += 1
            real_stops.append(shifts[temp_pos])
            break
    gene +=1
real_starts = np.unique(real_starts)
real_stops = np.unique(real_stops)

if len(real_starts) != len(real_stops):
    np.append(real_stops, genome_size)




In [50]:
for gene in range(n_genes + 1):
    gene_start = forward_genes[gene][3]
    while gene_start < shifts[pos]:
        pos += 1
        
        

IndexError: index 4272289 is out of bounds for axis 0 with size 4272289

In [56]:
shifts[150:170]

array([150, 151, 152, 153, 154, 416, 417, 418, 419, 420, 421, 422, 423,
       424, 425, 472, 842, 922, 923, 924])